In [21]:
import numpy as np
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [22]:
data = np.genfromtxt('Lomax.csv', delimiter=',', skip_header=1)

KeyboardInterrupt: 

In [3]:
n, Size = data.shape
train = data[:int(n*0.8), :]
test = data[int(n*0.8):, :]
test = test[test[:, 0].argsort()]

In [4]:
train_X = train[:, 2:]
train_Y = train[:, 0:2]
test_X = test[:, 2:]
test_Y = test[:, 0:2]

### Регрессия на интервалах

In [ ]:
lomax_inter = CatBoostRegressor(loss_function='MultiRMSE')
lomax_inter.fit(train_X, train_Y, eval_set = (test_X, test_Y))

0:	learn: 0.5158679	test: 0.5119198	best: 0.5119198 (0)	total: 2.18s	remaining: 36m 21s
1:	learn: 0.5130238	test: 0.5093218	best: 0.5093218 (1)	total: 5.08s	remaining: 42m 17s
2:	learn: 0.5103228	test: 0.5068499	best: 0.5068499 (2)	total: 8.21s	remaining: 45m 28s
3:	learn: 0.5075121	test: 0.5044192	best: 0.5044192 (3)	total: 11.2s	remaining: 46m 17s
4:	learn: 0.5049248	test: 0.5019809	best: 0.5019809 (4)	total: 14.1s	remaining: 46m 41s
5:	learn: 0.5023568	test: 0.4996339	best: 0.4996339 (5)	total: 16.7s	remaining: 46m 11s
6:	learn: 0.4998141	test: 0.4971937	best: 0.4971937 (6)	total: 19.4s	remaining: 45m 55s
7:	learn: 0.4974395	test: 0.4950629	best: 0.4950629 (7)	total: 22.1s	remaining: 45m 42s
8:	learn: 0.4950466	test: 0.4929232	best: 0.4929232 (8)	total: 24.7s	remaining: 45m 22s
9:	learn: 0.4927382	test: 0.4908202	best: 0.4908202 (9)	total: 27.5s	remaining: 45m 18s
10:	learn: 0.4904822	test: 0.4886692	best: 0.4886692 (10)	total: 30.2s	remaining: 45m 16s
11:	learn: 0.4883116	test: 0.4

In [6]:
Cat_predictions = lomax_inter.predict(test_X)

In [7]:
lomax_inter.save_model('lomax_inter_model.cbm')

In [ ]:
#Оценка качества регрессии первого параметра
mape = np.mean(np.abs((test_Y[:,0] - Cat_predictions[:,0]) / test_Y[:,0])) * 100
r2 = r2_score(test_Y[:,0], Cat_predictions[:,0])
mse = mean_squared_error(test_Y[:,0], Cat_predictions[:,0])
rmse = np.sqrt(mean_squared_error(test_Y[:,0], Cat_predictions[:,0]))
print("\nInter par 1")
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R² Score: {r2}')
print(f'MAPE: {mape}%')

#Оценка качества регрессии второго параметра
mape = np.mean(np.abs((test_Y[:,1] - Cat_predictions[:,1]) / test_Y[:,1])) * 100
r2 = r2_score(test_Y[:,1], Cat_predictions[:,1])
mse = mean_squared_error(test_Y[:,1], Cat_predictions[:,1])
rmse = np.sqrt(mean_squared_error(test_Y[:,1], Cat_predictions[:,1]))
print("\nInter par 2")
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R² Score: {r2}')
print(f'MAPE: {mape}%')


Inter par 1
MSE: 0.1250651692214374
RMSE: 0.3536455417808026
R² Score: 0.3248187369366101
MAPE: 13.767575372685883%

Inter par 2
MSE: 0.011133208020442681
RMSE: 0.10551401812291426
R² Score: 0.8599361913331927
MAPE: 23.124235980974028%


### Регрессия на статистиках

In [9]:
stat_M = np.zeros((7999,10)) # Матрицы для записиси статистик наблюдений
test_stat_M = np.zeros((2000,10))

In [10]:
#Далее для каждого наблюдения вычисляются 4 статистики
stat_M[:,0] = np.mean(train_X, axis=1) # Вычисляется среднее 
stat_M[:,1] = np.var(train_X, axis=1) # Вычисляется дисперсию 
stat_M[:,2] = np.std(train_X, axis=1) # Вычисляется стандартное отклонение
stat_M[:,3] =  stat_M[:,2] / stat_M[:,0] # Вычисляется вариацию 

quantiles = [0.1, 0.25, 0.5, 0.75, 0.9, 0.95]
for i, q in enumerate(quantiles):
    stat_M[:, 4 + i] = np.quantile(train_X, q, axis=1)

In [11]:
test_stat_M[:,0] = np.mean(test_X, axis=1)
test_stat_M[:,1] = np.var(test_X, axis=1)
test_stat_M[:,2] = np.std(test_X, axis=1)
test_stat_M[:,3] =  test_stat_M[:,2] / test_stat_M[:,0]

quantiles = [0.1, 0.25, 0.5, 0.75, 0.9, 0.95]
for i, q in enumerate(quantiles):
    test_stat_M[:, 4 + i] = np.quantile(test_X, q, axis=1)

In [ ]:
lomax_stat = CatBoostRegressor(iterations=10000,
                          learning_rate=0.05,
                          depth=5, loss_function='MultiRMSE')
lomax_stat.fit(stat_M, train_Y, eval_set = (test_stat_M, test_Y))

0:	learn: 0.5002421	test: 0.4961117	best: 0.4961117 (0)	total: 2.09ms	remaining: 20.9s
1:	learn: 0.4821205	test: 0.4781885	best: 0.4781885 (1)	total: 3.29ms	remaining: 16.5s
2:	learn: 0.4651600	test: 0.4615250	best: 0.4615250 (2)	total: 4.54ms	remaining: 15.1s
3:	learn: 0.4497047	test: 0.4463518	best: 0.4463518 (3)	total: 5.75ms	remaining: 14.4s
4:	learn: 0.4353995	test: 0.4322604	best: 0.4322604 (4)	total: 6.74ms	remaining: 13.5s
5:	learn: 0.4207995	test: 0.4178464	best: 0.4178464 (5)	total: 7.93ms	remaining: 13.2s
6:	learn: 0.4075203	test: 0.4048251	best: 0.4048251 (6)	total: 9.01ms	remaining: 12.9s
7:	learn: 0.3944161	test: 0.3920073	best: 0.3920073 (7)	total: 10.1ms	remaining: 12.6s
8:	learn: 0.3829288	test: 0.3807161	best: 0.3807161 (8)	total: 11.3ms	remaining: 12.5s
9:	learn: 0.3715467	test: 0.3696394	best: 0.3696394 (9)	total: 12.4ms	remaining: 12.3s
10:	learn: 0.3605599	test: 0.3588561	best: 0.3588561 (10)	total: 13.9ms	remaining: 12.6s
11:	learn: 0.3503499	test: 0.3488244	best

In [13]:
lomax_stat.save_model('lomax_stat_model.cbm')

In [14]:
Cat_predictions = lomax_stat.predict(test_stat_M)

In [15]:
#Оценка качества регрессии первого параметра
mape = np.mean(np.abs((test_Y[:,0] - Cat_predictions[:,0]) / test_Y[:,0])) * 100
r2 = r2_score(test_Y[:,0], Cat_predictions[:,0])
mse = mean_squared_error(test_Y[:,0], Cat_predictions[:,0])
rmse = np.sqrt(mean_squared_error(test_Y[:,0], Cat_predictions[:,0]))
print("\nStat par 1")
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R² Score: {r2}')
print(f'MAPE: {mape}%')

#Оценка качества регрессии второго параметра
mape = np.mean(np.abs((test_Y[:,1] - Cat_predictions[:,1]) / test_Y[:,1])) * 100
r2 = r2_score(test_Y[:,1], Cat_predictions[:,1])
mse = mean_squared_error(test_Y[:,1], Cat_predictions[:,1])
rmse = np.sqrt(mean_squared_error(test_Y[:,1], Cat_predictions[:,1]))
print("\nStat par 2")
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R² Score: {r2}')
print(f'MAPE: {mape}%')


Stat par 1
MSE: 0.008591704638859316
RMSE: 0.09269144857460863
R² Score: 0.9536165182836671
MAPE: 3.0445054189791962%

Stat par 2
MSE: 0.0008031844074294838
RMSE: 0.028340508242257825
R² Score: 0.9898953592747212
MAPE: 4.526015216533241%


### Регрессия на моментах

In [16]:
train_m_X = train_X.copy()
for i in range(train_X.shape[0]):
    for j in range(1, train_X.shape[1]):
        train_m_X[i,j] += train_m_X[i,j-1]

test_m_X = test_X.copy()
for i in range(test_X.shape[0]):
    for j in range(1, test_X.shape[1]):
        test_m_X[i,j] += test_m_X[i,j-1]

In [ ]:
lomax_m = CatBoostRegressor(loss_function='MultiRMSE')
lomax_m.fit(train_m_X, train_Y, eval_set = (test_m_X, test_Y))

0:	learn: 0.5120344	test: 0.5078807	best: 0.5078807 (0)	total: 2.44s	remaining: 40m 34s
1:	learn: 0.5055576	test: 0.5015490	best: 0.5015490 (1)	total: 4.66s	remaining: 38m 43s
2:	learn: 0.4994575	test: 0.4955475	best: 0.4955475 (2)	total: 6.81s	remaining: 37m 43s
3:	learn: 0.4935956	test: 0.4897491	best: 0.4897491 (3)	total: 8.94s	remaining: 37m 5s
4:	learn: 0.4880470	test: 0.4843038	best: 0.4843038 (4)	total: 10.9s	remaining: 36m 9s
5:	learn: 0.4828148	test: 0.4791391	best: 0.4791391 (5)	total: 12.9s	remaining: 35m 41s
6:	learn: 0.4776946	test: 0.4741516	best: 0.4741516 (6)	total: 15s	remaining: 35m 21s
7:	learn: 0.4728661	test: 0.4694141	best: 0.4694141 (7)	total: 17.3s	remaining: 35m 46s
8:	learn: 0.4682533	test: 0.4649017	best: 0.4649017 (8)	total: 19.5s	remaining: 35m 49s
9:	learn: 0.4638662	test: 0.4606089	best: 0.4606089 (9)	total: 21.7s	remaining: 35m 47s
10:	learn: 0.4596364	test: 0.4564613	best: 0.4564613 (10)	total: 23.8s	remaining: 35m 44s
11:	learn: 0.4556559	test: 0.45255

In [18]:
Cat_predictions = lomax_m.predict(test_X)

In [19]:
lomax_m.save_model('levi_moment_model.cbm')

In [ ]:
#Оценка качества регрессии первого параметра
mape = np.mean(np.abs((test_Y[:,0] - Cat_predictions[:,0]) / test_Y[:,0])) * 100
r2 = r2_score(test_Y[:,0], Cat_predictions[:,0])
mse = mean_squared_error(test_Y[:,0], Cat_predictions[:,0])
rmse = np.sqrt(mean_squared_error(test_Y[:,0], Cat_predictions[:,0]))
print("\nMoment par 1")
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R² Score: {r2}')
print(f'MAPE: {mape}%')

#Оценка качества регрессии второго параметра
mape = np.mean(np.abs((test_Y[:,1] - Cat_predictions[:,1]) / test_Y[:,1])) * 100
r2 = r2_score(test_Y[:,1], Cat_predictions[:,1])
mse = mean_squared_error(test_Y[:,1], Cat_predictions[:,1])
rmse = np.sqrt(mean_squared_error(test_Y[:,1], Cat_predictions[:,1]))
print("\nMoment par 2")
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R² Score: {r2}')
print(f'MAPE: {mape}%')


Moment par 1
MSE: 0.20306497384536162
RMSE: 0.450627311472975
R² Score: -0.09627377773014856
MAPE: 19.01199139280957%

Moment par 2
MSE: 0.32031652672479355
RMSE: 0.5659651285413206
R² Score: -3.029813565830943
MAPE: 93.73791157589308%
